In [2]:
import h5py
import numpy as np
# import matplotlib.pyplot as plt
from tifffile import imwrite, imread, tiffcomment, TiffFile
from ome_types import from_tiff
from glob import glob
import os
import affine_matrices as affine
from scipy.ndimage import affine_transform
from timeit import default_timer as timer
from npy2bdv import BdvWriter
from napari import Viewer
import lxml
from datetime import datetime


In [3]:
## define affine matrices for deskewing

def deskew(opm_angle, size_x, size_z):
    matrix_1_iso = affine.scale_matrix(sz=size_z/size_x)  # scale like in MVR calibration
    matrix_2_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    matrix_3_scale = affine.scale_matrix(sz=np.sin(opm_angle))  # scale before shear to preserve aspect ratio
    matrix_4_shear = affine.skew_matrix(opm_angle)  # shear to deskew

    total_matrix = matrix_4_shear @ matrix_3_scale @ matrix_2_rot @ matrix_1_iso
    return total_matrix

def just_rotate_90():
    matrix_2_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    return matrix_2_rot

In [ ]:
root_dir = "E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0/dOPM_t0000_p0000_z0000_c0000_view1/HamamatsuHam_DCAM-dopm/"

stack_path = os.path.join(root_dir, "HamamatsuHam_DCAM-dopm_MMStack.ome.tif")


omexml = from_tiff(stack_path)
iminfo = omexml.images[0]

size_x = iminfo.pixels.size_x
size_y = iminfo.pixels.size_y
size_z = len(omexml.images)
vox_size_x_um = iminfo.pixels.physical_size_x
vox_size_y_um = iminfo.pixels.physical_size_y
vox_size_z_um = iminfo.pixels.physical_size_z

print("size_x", size_x)
print("size_y", size_y)
print("size_z", size_z)

imstack = np.zeros((size_x, size_y, 403), dtype=np.uint16)

meta = tiffcomment(stack_path)
print(meta)
with TiffFile(stack_path) as tif:
    # summary_metadata = tif.imagej_metadata

    # equivalent of getImageInfo() in ImageJ but in Python
    mm_metadata = tif.micromanager_metadata
    print(tif.imagej_metadata)
    
    #for n, page in enumerate(tif.pages):
        # print("page", n)
        #print(dir(page))
        # raise
        # imstack[:,:,n] = page.asarray().T # transpose for x->y y->x

# print(ome_metadata)
# parse ome_metadata with lxml
mm_metadata.keys()

#for elem in xml_ome_metadata.iter():
#    print("tag", elem.tag, "text", elem.text)

In [ ]:
laser = mm_metadata['Summary']['UserData']['laser']
filter = mm_metadata['Summary']['UserData']['filter']

### Try to combine the channels, views and timepoints into a single H5 file
Currently don't handle Z slices or times correctly -- todo.

In [3]:
root_dir = "A:/leo/micromanager/bone_marrow_2nd/aml/data"
# root_dir = "A:/leo/micromanager/bonemarrow/data/"
# root_dir = "A:/leo/micromanager/dan/D_and_E/data/"
# root_dir = "A:/leo/micromanager/dan/D_and_E_frome4/data/"

# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241216-171205_c0p0/")
tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20250117-181620")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241209-144142")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20250116-160803")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20250116-172759")

folder_name = "HamamatsuHam_DCAM-dopm"

OPM_ANGLE = 45  # opm angle in degrees

tiff_dirs = glob(os.path.join(tiff_data_dir, "dOPM*"))  # unique dirs for each MDA dimension e.g. channel, time, z
print(tiff_dirs)

# what to do with z slices
only_keep_z0 = True

# group dirs with the same position pattern i.e. p0000, p0001, p0002
grouped_dirs = {}
grouped_details = {}

for tiff_dir in tiff_dirs:
    print(tiff_dir)
    view_details = os.path.basename(tiff_dir).split("_")
    angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
    position_key = view_details[2]
    position_str = position_key[1:]  # remove 'p' from 'p0000'
    time_str = view_details[1][1:]  # remove 't' from 't0000'
    z_str = view_details[3][1:]  # remove 'z' from 'z0000'
    channel_str = view_details[4][1:]  # remove 'c' from 'c0000'

    tiff_dir_dict = {}
    tiff_dir_dict["time"] = time_str
    tiff_dir_dict["z"] = z_str
    tiff_dir_dict["channel"] = channel_str
    tiff_dir_dict["dir"] = tiff_dir
    tiff_dir_dict["angle"] = angle

    if only_keep_z0 and int(z_str)>0:
        continue

    if position_key in grouped_dirs:
        grouped_dirs[position_key].append(tiff_dir)
        grouped_details[position_key].append(tiff_dir_dict)
    else:
        grouped_dirs[position_key] = [tiff_dir]
        grouped_details[position_key] = [tiff_dir_dict]
    
print(grouped_dirs)
print(grouped_details)
# unique_positions = list(grouped_dirs.keys())

for p, d in grouped_details.items():
    # number of channels in position (extract only 'channel' item from each dict)
    channels = list(set([int(i["channel"]) for i in d]))
    # do previous line in a neater way
    

    print(channels)

# now do the same thing for time and z
# then iterate over all combinations of positions, times and z to get the full list of files to process



['A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0000_z0000_c0000_view1', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0000_z0000_c0000_view2', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0000_z0000_c0001_view1', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0000_z0000_c0001_view2', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0001_z0000_c0000_view1', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0001_z0000_c0000_view2', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0001_z0000_c0001_view1', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000_p0001_z0000_c0001_view2', 'A:/leo/micromanager/bone_marrow_2nd/aml/data\\y_stage_scan_20250117-181620\\dOPM_t0000

In [ ]:
omexml.images[0]

In [21]:
grouped_details
for position, details in grouped_details.items():
    print(position)
    print(details)
    print([d["angle"] + OPM_ANGLE for d in details])

p0000
[{'time': '0000', 'z': '0000', 'channel': '0000', 'dir': 'A:/leo/micromanager/dan/D_and_E/data/y_stage_scan_20250116-160803\\dOPM_t0000_p0000_z0000_c0000_view1', 'angle': -45}, {'time': '0000', 'z': '0000', 'channel': '0001', 'dir': 'A:/leo/micromanager/dan/D_and_E/data/y_stage_scan_20250116-160803\\dOPM_t0000_p0000_z0000_c0001_view1', 'angle': -45}]
[0, 0]
p0001
[{'time': '0000', 'z': '0000', 'channel': '0000', 'dir': 'A:/leo/micromanager/dan/D_and_E/data/y_stage_scan_20250116-160803\\dOPM_t0000_p0001_z0000_c0000_view1', 'angle': -45}, {'time': '0000', 'z': '0000', 'channel': '0001', 'dir': 'A:/leo/micromanager/dan/D_and_E/data/y_stage_scan_20250116-160803\\dOPM_t0000_p0001_z0000_c0001_view1', 'angle': -45}]
[0, 0]
p0002
[{'time': '0000', 'z': '0000', 'channel': '0000', 'dir': 'A:/leo/micromanager/dan/D_and_E/data/y_stage_scan_20250116-160803\\dOPM_t0000_p0002_z0000_c0000_view1', 'angle': -45}, {'time': '0000', 'z': '0000', 'channel': '0001', 'dir': 'A:/leo/micromanager/dan/D_an

In [4]:
# get current date for file timestamp
now = datetime.now()
date_time = now.strftime("%Y%m%d-%H%M%S")
print(date_time)

out_dir = os.path.join(root_dir, f"deskewed_{date_time}/")

if os.path.exists(out_dir):
    print("Output directory already exists, exiting")
    raise(FileExistsError)

# npos to do
stop_early = False
npos_max = 5

i_p = 0
# loop over grouped dirs to debug
for position, details in grouped_details.items():
    if stop_early and i_p>npos_max:
        break
    i_p+=1
    dirs = [d["dir"] for d in details]
    channels_strs = [d["channel"] for d in details]
    time_strs = [d["time"] for d in details]
    z_strs = [d["z"] for d in details]
    angles = [d["angle"] + OPM_ANGLE for d in details]
    angle_idxc = [0 if a == 0 else 1 for a in angles]

    # print all those dimenisons e.g. time, z, channel in a single print statement
    print("channels", channels_strs, "time", time_strs, "z", z_strs, "angle", angles)

    # to get n channels etc. use set(channels) etc. to get unique values
    n_channels = len(set(channels_strs))
    n_times = len(set(time_str))
    n_z = len(set(z_str)) # if not only_keep_z0 else 1
    n_angles = len(set(angles))

    print("n_channels", n_channels, "n_times", n_times, "n_z", n_z, "n_angles", n_angles)

    # for BDV dataset attributes
    channel_attr = [None]*n_channels
    angle_attr = [None]*n_angles

    # create BDV dataset
    save_path = os.path.join(out_dir, "pos_"+position)
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, "dataset.h5")

    bdv_writer = BdvWriter(save_file, 
                           nchannels=n_channels, 
                           nilluminations=1, 
                           nangles=n_angles,
                           ntiles= n_z)

    for n, tiff_dir in enumerate(dirs):
        print(tiff_dir)
        view_details = os.path.basename(tiff_dir).split("_")
        angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
        angle_i = angle_idxc[n]
        position_str = position
        position_i = int(position_str[1:])
        time_str = time_strs[n]
        time_i = int(time_str)
        z_str = z_strs[n]
        z_i = int(z_str)
        channel_str = channels_strs[n]
        channel_i = int(channel_str)

        print("angle", angle)
        print("position", position_str)
        print("time", time_str)
        print("z", z_str)
        print("channel", channel_str)

        # print(stack_dir)
        print(os.path.join(tiff_dir, "HamamatsuHam_DCAM-dopm/*.tif"))
        stacks = glob(os.path.join(tiff_dir, "HamamatsuHam_DCAM-dopm/*.tif"))

        # get ome metadata using ome-types library, tifffile doesnt parse xml into a dict unlike mm metadata
        omexml = from_tiff(stacks[0])
        iminfo = omexml.images[0]

        size_x = iminfo.pixels.size_x
        size_y = iminfo.pixels.size_y
        size_z = len(omexml.images)
        vox_size_x_um = iminfo.pixels.physical_size_x
        vox_size_y_um = iminfo.pixels.physical_size_y
        vox_size_z_um = iminfo.pixels.physical_size_z
        
        print("size_x", size_x)
        print("size_y", size_y)
        print("size_z", size_z)

        imstack = np.zeros((size_z, size_y, size_x), dtype=np.uint16)

        start_time = timer()

        # read the mm-split 4gb tiff stacks into one big stack (matrix), think about memory concerns...
        
        offset = 0
        for i in range(len(stacks)):
            print("loading",  stacks[i])
            with TiffFile(stacks[i]) as tif:
                mm_metadata = tif.micromanager_metadata
                for n, page in enumerate(tif.pages):
                    # print("page", n)
                    imstack[n+offset,:,:] = page.asarray() # expects zyx
            offset += n+1

        end_time = timer()
        print("Read time", (end_time-start_time), "ms")

        laser_meta = mm_metadata['Summary']['UserData']['laser']
        filter_meta = mm_metadata['Summary']['UserData']['filter']
        
        channel_attr[channel_i] = f"{laser_meta}"
        angle_attr[angle_i] = f"{angle}"

        start_time = timer()  # start timer for saving h5 file

        angle_from_zero = angle+OPM_ANGLE
        print("angle", angle_i, "time",time_i,"tile",z_i,"channel",channel_i)
        bdv_writer.append_view(
            imstack, angle=angle_i, time=time_i, tile=z_i, channel=channel_i,
            calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
            voxel_size_xyz=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
            voxel_units='micron')
        print(f"dataset in {save_file}")
        bdv_writer.write_xml()

    # loop again to append the affines... annoying to have to do this
    for n in range(len(dirs)):
        angle_i = angle_idxc[n]
        position_i = int(position_str[1:])
        time_i = int(time_strs[n])
        z_i = int(z_strs[n])
        channel_i = int(channels_strs[n])

        bdv_writer.append_affine(
            affine.rotation_matrix(theta=np.pi/2, axis='x', square=False),
            name_affine='90 deg rotation about x',
            angle=angle_i, time=time_i, tile=z_i, channel=channel_i)
        skew_angle = -np.pi*(angle)/180
        bdv_writer.append_affine(
            affine.scale_matrix(sz=np.sin(skew_angle), square=False),
            name_affine='scale before shear',
            angle=angle_i, time=time_i, tile=z_i, channel=channel_i)
        bdv_writer.append_affine(
            affine.skew_matrix(skew_angle, square=False),
            name_affine='shear to deskew',
            angle=angle_i, time=time_i, tile=z_i, channel=channel_i)

        
    # 
    bdv_writer.set_attribute_labels('angle', angle_attr) 
    bdv_writer.set_attribute_labels('channel', channel_attr)
    """
    bdv_writer.append_affine(
        affine.rotation_matrix(theta=np.pi/2, axis='x', square=False),
        name_affine='90 deg rotation about x')
    skew_angle = -np.pi*(angle)/180
    bdv_writer.append_affine(
        affine.scale_matrix(sz=np.sin(skew_angle), square=False),
        name_affine='scale before shear')
    bdv_writer.append_affine(
        affine.skew_matrix(skew_angle, square=False),
        name_affine='shear to deskew')
    bdv_writer.set_attribute_labels('angle', angle_attr) 
    bdv_writer.set_attribute_labels('channel', channel_attr)
    """
    bdv_writer.close()

20250119-160952
channels ['0000', '0000', '0001', '0001'] time ['0000', '0000', '0000', '0000'] z ['0000', '0000', '0000', '0000'] angle [0, 90, 0, 90]
n_channels 2 n_times 1 n_z 1 n_angles 2
A:/leo/micromanager/bone_marrow_2nd/aml/data\y_stage_scan_20250117-181620\dOPM_t0000_p0000_z0000_c0000_view1
angle -45
position p0000
time 0000
z 0000
channel 0000
A:/leo/micromanager/bone_marrow_2nd/aml/data\y_stage_scan_20250117-181620\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCAM-dopm/*.tif
size_x 2304
size_y 2304
size_z 1000
loading A:/leo/micromanager/bone_marrow_2nd/aml/data\y_stage_scan_20250117-181620\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCAM-dopm\HamamatsuHam_DCAM-dopm_MMStack.ome.tif
loading A:/leo/micromanager/bone_marrow_2nd/aml/data\y_stage_scan_20250117-181620\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCAM-dopm\HamamatsuHam_DCAM-dopm_MMStack_1.ome.tif
loading A:/leo/micromanager/bone_marrow_2nd/aml/data\y_stage_scan_20250117-181620\dOPM_t0000_p0000_z0000_c0000

IndexError: index out of range

In [13]:



tiff_dirs = glob(os.path.join(tiff_data_dir, "dOPM*"))
for tiff_dir in tiff_dirs:
    tiff_dir_basename = os.path.basename(tiff_dir)
    stack_dir = os.path.join(tiff_data_dir, tiff_dir, folder_name)  # dir containg the tiff stacks
    # print(stack_dir)
    stacks = glob(os.path.join(stack_dir, "*.tif"))
    print(stacks)
    omexml = from_tiff(stacks[0])
    iminfo = omexml.images[0]

    # find view1 or view2 pattern in string tiff_dir
    angle = -OPM_ANGLE if "view1" in tiff_dir_basename else OPM_ANGLE
    angle_i = 0 if "view1" in tiff_dir_basename else 1
    angle *= np.pi/180  # convert to radians

    # find pattern in string tiff_dir
    example_str = "dOPM_t0000_p0000_z0000_c0000_view1"

    # get view details like time (t), position (p), z-slice (z), channel (c)
    # according to example_str
    view_details = tiff_dir_basename.split("_")
    angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
    position_str = view_details[2][1:]  # remove 'p' from 'p0000'
    time_str = view_details[1][1:]  # remove 't' from 't0000'
    z_str = view_details[3][1:]  # remove 'z' from 'z0000'
    channel_str = view_details[4][1:]  # remove 'c' from 'c0000'

    # convert string to int
    position = int(position_str)
    time = int(time_str)
    z = int(z_str)
    channel = int(channel_str)
 
    size_x = iminfo.pixels.size_x
    size_y = iminfo.pixels.size_y
    size_z = len(omexml.images)
    vox_size_x_um = iminfo.pixels.physical_size_x
    vox_size_y_um = iminfo.pixels.physical_size_y
    vox_size_z_um = iminfo.pixels.physical_size_z

    print("size_x", size_x)
    print("size_y", size_y)
    print("size_z", size_z)

    # affine matrices
    matrix_1_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    matrix_2_prescale = affine.scale_matrix(sz=np.sin(angle))  # scale before shear to preserve aspect ratio
    matrix_3_shear = affine.skew_matrix(angle)  # shear to deskew

    imstack = np.zeros((size_z, size_y, size_x), dtype=np.uint16)

    start_time = timer()

    # read the mm-split 4gb tiff stacks into one big stack (matrix), think about memory concerns...
    offset = 0
    for i in range(len(stacks)):
        print("loading",  stacks[i])
        with TiffFile(stacks[i]) as tif:
            print(tif.pages)
            for n, page in enumerate(tif.pages):
                # print("page", n)
                imstack[n+offset,:] = page.asarray() # npy2bdv expects zyx order
        offset += n+1

    end_time = timer()
    print("Read time", (end_time-start_time), "ms")

    # create BDV dataset
    save_path = os.path.join(out_dir, "pos_"+position_str+"_time_"+time_str+"_z_"+z_str+"_channel_"+channel_str)
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, "dataset.h5")

    n = 1
    while os.path.exists(save_file):
        save_file = os.path.join(save_path, f"dataset-{n}.h5")
        n += 1
    try:
        bdv_writer = BdvWriter(save_file, nchannels=1, nilluminations=1, nangles=2)


        #imstack = imread(stacks[0])
        print(np.shape(imstack))
        start_time = timer()

        angle_from_zero = angle+OPM_ANGLE
        bdv_writer.append_view(imstack, angle=angle_i,
                calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
                voxel_size_xyz=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
                voxel_units='um')
        bdv_writer.set_attribute_labels('angle', str(angle_from_zero))
        bdv_writer.write_xml()
        bdv_writer.append_affine(
            affine.rotation_matrix(theta=np.pi/2, axis='x', square=False),
            name_affine='90 deg rotation about x')
        skew_angle = np.pi*(angle)/180
        bdv_writer.append_affine(
            affine.scale_matrix(sz=np.sin(skew_angle), square=False),
            name_affine='scale before shear')
        bdv_writer.append_affine(
            affine.skew_matrix(skew_angle, square=False),
            name_affine='shear to deskew')
        # bdv_writer.write_xml()
        bdv_writer.close()
        print(f"dataset in {save_file}")
    except Exception as e:
        del bdv_writer  # release from memory
        raise(e)
    
    


['A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack.ome.tif', 'A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_1.ome.tif', 'A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_2.ome.tif', 'A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_3.ome.tif', 'A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_4.ome.tif', 'A:/leo/micromanager/dan/B2_B3_B4_b5/data/y_stage_scan_20250115-123649\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-

AssertionError: Length of labels 1 must match the number of attributes 2

In [ ]:
root_dir = "E:/dopm_reslicing/"
tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241216-171205_c0p0/")
out_dir = os.path.join(root_dir, "y_stage_scan_20241216-171205_c0p0/deskewedtest/")
folder_name = "HamamatsuHam_DCAM-dopm"

OPM_ANGLE = 45  # opm angle in degrees

tiff_dirs = glob(os.path.join(tiff_data_dir, "dOPM*"))
for tiff_dir in tiff_dirs:
    tiff_dir_basename = os.path.basename(tiff_dir)
    stack_dir = os.path.join(tiff_data_dir, tiff_dir, folder_name)  # dir containg the tiff stacks
    # print(stack_dir)
    stacks = glob(os.path.join(stack_dir, "*.tif"))
    print(stacks)
    omexml = from_tiff(stacks[0])
    iminfo = omexml.images[0]

    # find view1 or view2 pattern in string tiff_dir
    angle = -OPM_ANGLE if "view1" in tiff_dir_basename else OPM_ANGLE
    angle_i = 0 if "view1" in tiff_dir_basename else 1
    angle *= np.pi/180  # convert to radians

    # find pattern in string tiff_dir
    example_str = "dOPM_t0000_p0000_z0000_c0000_view1"

    # get view details like time (t), position (p), z-slice (z), channel (c)
    # according to example_str
    view_details = tiff_dir_basename.split("_")
    angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
    position_str = view_details[2][1:]  # remove 'p' from 'p0000'
    time_str = view_details[1][1:]  # remove 't' from 't0000'
    z_str = view_details[3][1:]  # remove 'z' from 'z0000'
    channel_str = view_details[4][1:]  # remove 'c' from 'c0000'

    # convert string to int
    position = int(position_str)
    time = int(time_str)
    z = int(z_str)
    channel = int(channel_str)
 
    size_x = iminfo.pixels.size_x
    size_y = iminfo.pixels.size_y
    size_z = len(omexml.images)
    vox_size_x_um = iminfo.pixels.physical_size_x
    vox_size_y_um = iminfo.pixels.physical_size_y
    vox_size_z_um = iminfo.pixels.physical_size_z

    print("size_x", size_x)
    print("size_y", size_y)
    print("size_z", size_z)

    # affine matrices
    matrix_1_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    matrix_2_prescale = affine.scale_matrix(sz=np.sin(angle))  # scale before shear to preserve aspect ratio
    matrix_3_shear = affine.skew_matrix(angle)  # shear to deskew

    imstack = np.zeros((size_x, size_y, size_z), dtype=np.uint16)

    start_time = timer()

    # read the mm-split 4gb tiff stacks into one big stack (matrix), think about memory concerns...
    offset = 0
    for i in range(len(stacks)):
        print("loading",  stacks[i])
        with TiffFile(stacks[i]) as tif:
            print(tif.pages)
            for n, page in enumerate(tif.pages):
                # print("page", n)
                imstack[:,:,n+offset] = page.asarray().T # transpose for x->y y->x
        offset += n+1

    end_time = timer()
    print("Read time", (end_time-start_time), "ms")

    # create BDV dataset
    save_path = os.path.join(out_dir, "pos_"+position_str+"_time_"+time_str+"_z_"+z_str+"_channel_"+channel_str)
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, "dataset.h5")

    n = 1
    while os.path.exists(save_file):
        save_file = os.path.join(save_path, f"dataset-{n}.h5")
        n += 1
    try:
        bdv_writer = BdvWriter(save_file, nchannels=1, nilluminations=1, nangles=2)


        #imstack = imread(stacks[0])
        print(np.shape(imstack))
        start_time = timer()

        angle_from_zero = angle+OPM_ANGLE
        bdv_writer.append_view(imstack, angle=angle_i,
                calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
                voxel_size_xyz=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
                voxel_units='um')
        bdv_writer.set_attribute_labels('angle', str(angle_from_zero))
        bdv_writer.set_attribute_labels('channel', "hello")
        bdv_writer.set_attribute_labels('channel', "world")
        bdv_writer.write_xml()
        bdv_writer.append_affine(
            affine.rotation_matrix(theta=np.pi/2, axis='x', square=False),
            name_affine='90 deg rotation about x')
        skew_angle = np.pi*(angle)/180
        bdv_writer.append_affine(
            affine.scale_matrix(sz=np.sin(skew_angle), square=False),
            name_affine='scale before shear')
        bdv_writer.append_affine(
            affine.skew_matrix(skew_angle, square=False),
            name_affine='shear to deskew')
        # bdv_writer.write_xml()
        bdv_writer.close()
        print(f"dataset in {save_file}")
    except Exception as e:
        del bdv_writer  # release from memory
        raise(e)
    
    


In [4]:
#del bdv_writer  # release from memory
bdv_writer.xm

In [ ]:
%%time

root_dir = "E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0/dOPM_t0000_p0000_z0000_c0000_view1/HamamatsuHam_DCAM-dopm/"
stack_path = os.path.join(root_dir, "HamamatsuHam_DCAM-dopm_MMStack.ome.tif")

nz, ny, nx = 403, 2304, 2304

bdv_savedir = os.path.join(root_dir, "bdvtest")
os.makedirs(bdv_savedir, exist_ok=True)

fname = os.path.join(bdv_savedir, "stack.h5")
bdv_writer = BdvWriter(fname, nchannels=1, nilluminations=1, nangles=1)

# bdv_writer.set_attribute_labels('channel', ('488', '561'))
# bdv_writer.set_attribute_labels('illumination', ('L', 'R'))
bdv_writer.set_attribute_labels('angle', ('0'))

imstack = np.zeros((size_x, size_y, 403), dtype=np.uint16)

omexml = from_tiff(stack_path)
iminfo = omexml.images[0]

size_x = iminfo.pixels.size_x
size_y = iminfo.pixels.size_y
size_z = len(omexml.images)
vox_size_x_um = iminfo.pixels.physical_size_x
vox_size_y_um = iminfo.pixels.physical_size_y
vox_size_z_um = iminfo.pixels.physical_size_z

with TiffFile(stack_path) as tif:
    print(tif.pages)
    for n, page in enumerate(tif.pages):
        # print("page", n)
        imstack[:,:,n] = page.asarray().T # transpose for x->y y->x

affine_matrix = just_rotate_90(square=False)
OPM_ANGLE = 45
angles = [0, 90]
for a in range(1):
    bdv_writer.append_view(imstack, angle=angles[a],
            calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um))
    bdv_writer.write_xml()
    bdv_writer.append_affine(
        affine.rotation_matrix(theta=np.pi/2, axis='x'),
        name_affine='90 deg rotation about x')
    skew_angle = np.pi*(angles[a]-OPM_ANGLE)/180
    bdv_writer.append_affine(
        affine.scale_matrix(sz=np.sin(skew_angle),
        name_affine='scale before shear'))
    bdv_writer.append_affine(
        affine.skew_matrix(skew_angle),
        name_affine='shear to deskew')
bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset in {fname}")



In [5]:


just_rot = just_rotate_90()
imstack_rot = affine_transform(imstack, just_rot)

In [13]:
# make imstack_rot (z,y,x) instead of (x,y,z)
imstack_rot2 = np.moveaxis(imstack_rot, [0,1,2], [2,1,0])
imstack_rot2.shape

imwrite("testrot.tif", imstack_rot2)

In [ ]:
viewer = Viewer()
viewer.add_image(imstack_rot)
viewer.show()